In [11]:
import pandas as pd

df = pd.read_csv("ScrapeDataFromTiktokNew.csv")
df.head()

,Username,Comment,Target
0,ANDY TORAJA,tapi sy heran knp tdk dari awal di persoalkan ...,0
1,Juan Felix Michael,jelas2.... pemungutan ulang dan 02 didiskualif...,0
2,818anakrantau,trus apa fungsi dkpp ketika ada pelanggaran tp...,1
3,ArayamirShop,"Yakin Pemilu ulang, KPU sdh jadwalkan bulan Ju...",0
4,Faizi,02 Nyungsep,0


In [36]:
duplicates = df[df.duplicated()]
print("Baris yang duplikat:")
print(len(duplicates))

Baris yang duplikat:
0


In [14]:
df.shape

(377, 3)